In [ ]:
#Instalacion del modulo scikit-surprise
!pip install scikit-surprise

# **LIBRERÍAS**

**PANDAS** Esta librería nos permite importar desde diferentes fuentes de datos, guardándose en un objeto de tipo data frame con el cual podemos realizar diferentes operaciones.

**NUMPY** Permite realizar funciones matemáticas con un alto nivel en la administración de matrices multidimensionales

**SKLEARN** Es una librería que cuenta con algoritmos que permiten hacer la agrupación de conjuntos de objetos sin etiquetar y estimar las relaciones que existe entre las variables.

**SCIPY** Es una biblioteca de Python que permite hacer la resolución de problemas ya sean matemáticos o científicos. Está construido sobre una extensión de NumPy y permite visualizar los datos con un alto nivel en diferentes comandos.

**SURPRISE** Esta librería permite realizar sistemas de recomendación para datos explícitos, ya que tiene muchas funciones avanzadas, permite implementar soluciones de manera rápida para nuestros modelos.


In [ ]:
#Importacion de las librerias necesarias para el ejemplo.
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

from surprise import Dataset
from surprise import Reader
from surprise import KNNWithMeans 
from surprise import accuracy
from surprise.model_selection import train_test_split

In [ ]:
# Constante con el nombre del dataset
PATH = 'data.csv'

# Importar datos

In [ ]:
#Importacion y lectura del dataset

df = pd.read_csv(PATH)
df.shape

(100000, 10)

In [ ]:
#Lectura de los cinco primeros registros del dataset.
df.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,655,52,4,11482,300,4,8,2012,94,7
1,2713,90,3,6479,469,1,8,2012,33,5
2,409,17,2,25472,435,1,12,2001,196,4
3,1150,234,10,23950,529,2,23,2019,79,2
4,2424,390,5,13046,395,2,20,2010,200,4


# Método 1

In [ ]:
# Generar una tabla dinámica con lectores en el índice y libros en la columna y valores que son las calificaciones
pt_df = df.pivot_table(
    columns = 'book_id',
    index = 'reader_id',
    values = 'book_rating'
).fillna(0)

# Coversion a una matriz CSR
mat = pt_df.values
mat = csr_matrix(mat)

In [ ]:
def normalize(pred_ratings):
    '''
    Esta función normalizará la entrada pred_ratings
    
    parametros:
        pred_ratings (Lista -lista >) : Las calificaciones de predicción
    '''
    return (pred_ratings - pred_ratings.min()) / (pred_ratings.max() - pred_ratings.min())

In [ ]:
def generate_prediction_df(mat, pt_df, n_factors):
    '''
        
    Esta función calculará la descomposición de un solo valor de la matriz de entrada
    dado n_factors. Luego generará y normalizará las predicciones de calificación del usuario.
    
    parametros:
            
    mat (CSR Matrix) : matriz scipy csr correspondiente a la tabla dinámica (pt_df)
        pt_df (DataFrame) : pandas dataframe que es una tabla dinámica
        n_factors (Integer) : Número de valores singulares y vectores a calcular. 
                              Debe ser 1 < = n_factors < min(mat.shape). 
    '''
    
    if not 1 <= n_factors < min(mat.shape):
        raise ValueError("Must be 1 <= n_factors < min(mat.shape)")
        
    # factorización matricial
    u, s, v = svds(mat, k = n_factors)
    s = np.diag(s)

    # calcular clasificaciones de pred
    pred_ratings = np.dot(np.dot(u, s), v) 
    pred_ratings = normalize(pred_ratings)
    
    # Convertir a df
    pred_df = pd.DataFrame(
        pred_ratings,
        columns = pt_df.columns,
        index = list(pt_df.index)
    ).transpose()
    return pred_df

In [ ]:
%time pred_df = generate_prediction_df(mat, pt_df, 10)

CPU times: user 1.27 s, sys: 794 ms, total: 2.06 s
Wall time: 1.36 s


In [ ]:
def recommend_items(pred_df, usr_id, n_recs):
    '''
        
      Dada una usr_id y pred_df esta función recomendará
        para el usuario.
    
    parametros:
        pred_df (DataFrame) : generado a partir de la función 'generate_prediction_df'
        usr_id (Integer) : El usuario para el que desea obtener recomendaciones de artículos
        n_recs (Integer) : El número de recomendaciones que desea para este usuario
    '''
    
    usr_pred = pred_df[usr_id].sort_values(ascending = False).reset_index().rename(columns = {usr_id : 'sim'})
    rec_df = usr_pred.sort_values(by = 'sim', ascending = False).head(n_recs)
    return rec_df

In [ ]:
recommend_items(pred_df, 5, 5)

,book_id,sim
0,1108,0.143244
1,1839,0.140945
2,725,0.140516
3,1897,0.140497
4,1341,0.140086


# **Método 2**

In [ ]:
pt_df = df.pivot_table(
    columns = 'book_id', 
    index = 'author_id', 
    values = 'book_rating'
).fillna(0)

In [ ]:
def find_similar_readers(pt_df, reader_id, n_recs):
    '''
     Esta función encontrará lectores similares al usuario especificado reader_id
    '''
    
    # lector separado de interés y todos los demás lectores 
    reader = pt_df[pt_df.index == reader_id]
    other_readers = pt_df[pt_df.index != reader_id]

    # obtener similitud del lector actual y todos los demás lectores
    sim = cosine_similarity(reader, other_readers)[0].tolist()
    idx = other_readers.index.tolist()

    # Cree un diccionario de similitud para este usuario W.R.T Todos los demás usuarios
    idx_sim = dict(zip(idx, sim))
    idx_sim = sorted(idx_sim.items(), key=lambda x: x[1], reverse=True)

    similar_readers = idx_sim[:n_recs]
    readers = [rdr[0] for rdr in similar_readers]

    return readers

In [ ]:
find_similar_readers(pt_df = pt_df, reader_id = 226, n_recs = 5)

[319, 191, 145, 162, 212]

# **Método 3**

In [ ]:
rdf = df[['reader_id', 'book_id', 'book_rating']]

In [ ]:
#Lectura del conjunto de datos
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(rdf, reader)

In [ ]:
#Dividir el conjunto de datos
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)

In [ ]:
# Utilice user_based true/false para cambiar entre el filtrado colaborativo basado en usuarios o en elementos
algo = KNNWithMeans(k=5, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [ ]:
# Ejecute el modelo entrenado en el conjunto de pruebas
test_pred = algo.test(testset)

In [ ]:
# obtener RMSE
accuracy.rmse(test_pred, verbose=True)

RMSE: 2.9306


2.9306185721359865

In [ ]:
algo.predict(uid = 10, iid = 43)

Prediction(uid=10, iid=43, r_ui=None, est=5, details={'actual_k': 0, 'was_impossible': False})

# **Interpretación de la métrica**
Con respecto a la medida de rendimiento de RMSE, se obtuvo como resultado 2.9306, este valor es bajo y por lo tanto indica que se tiene un mejor ajuste. Esta medida nos indica una buena precisión y por lo tanto el modelo predice la respuesta. Además con este valor nos indica que el modelo realiza buenas predicciones. Como se pudo observar la implementación generada realiza recomendaciones de artículos basadas en user_id. 
